In [ ]:
cd ..

In [ ]:
import os
import numpy as np

import util.io

In [ ]:
DATA_FILE = './exp-visual7w/visual7w-dataset/datasets/visual7w-pointing/dataset.json'
IMAGE_DIR = './exp-visual7w/visual7w-dataset/images/'
data = util.io.load_json(DATA_FILE)

In [ ]:
def _convert_bbox(old_bbox):
    x, y, w, h = old_bbox['x'], old_bbox['y'], old_bbox['width'], old_bbox['height']
    bbox = [x, y, x+w-1, y+h-1]
    return bbox

data_images = data['images']
data_bboxes = data['boxes']
bbox_map = {box['box_id']: _convert_bbox(box) for box in data_bboxes}

In [ ]:
# imdb_format:
#   a list of images
# each image:
#   a dict
#   { "image_path": "...",
#     "processed_qa_pairs": ([[x1, y1, x2, y2], ...], sentence, label),
#     "split": "train" }

# Here we use a different type of format:
imdb = []
for data_img in data_images:
    filename = data_img['filename']
    image_path = os.path.join(IMAGE_DIR, filename)
    split = data_img['split']
    
    processed_qa_pairs = []
    for qa_pair in data_img['qa_pairs']:
        # put the ground-truth bounding box at the beginning
        bboxes = [bbox_map[qa_pair['answer']]] + [bbox_map[box_id] for box_id in qa_pair['multiple_choices']]
        bboxes = np.array(bboxes)
        question = qa_pair['question']
        label = 0
        processed_qa_pairs.append((bboxes, question, label))
    
    iminfo = dict(im_path=image_path, processed_qa_pairs=processed_qa_pairs, split=split)
    imdb.append(iminfo)

In [ ]:
imdb_trn = [iminfo for iminfo in imdb if iminfo['split'] == 'train']
imdb_val = [iminfo for iminfo in imdb if iminfo['split'] == 'val']
imdb_tst = [iminfo for iminfo in imdb if iminfo['split'] == 'test']

In [ ]:
if not os.path.isdir('./exp-visual7w/data/imdb'):
    os.mkdir('./exp-visual7w/data/imdb')

In [ ]:
util.io.save_numpy_obj(imdb_trn, './exp-visual7w/data/imdb/imdb_trn.npy')
util.io.save_numpy_obj(imdb_val, './exp-visual7w/data/imdb/imdb_val.npy')
util.io.save_numpy_obj(imdb_tst, './exp-visual7w/data/imdb/imdb_tst.npy')